# Adequacy - MPA Size 4.1

### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

#### 4.1 MPA Size

In [5]:
# Intersect the MPAs with the study area

in_feature_1 = "WDPA_MPA"
in_feature_2 = "Caribbean_detailed_subregions_mol"
out_feature = "\\MPA_Carib_int_Adq_4_1"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseIntersect(
    in_features=[in_feature_1,in_feature_2],
    out_feature_class= output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_Carib_int_Adq_4_1'>

In [7]:
# Repair geoemtry on the intersect, calculate area, and export to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Adequacy"

in_feature = "MPA_Carib_int_Adq_4_1"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area_export():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features= in_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )
    
    # Export tables to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = in_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
repair_geo_area_export()


In [3]:
# Convert the output features to a Pandas DataFrame
df_MPA = pd.DataFrame.spatial.from_featureclass("MPA_Carib_int_Adq_4_1")


In [4]:

# Display the DataFrame
df_MPA.head()


,OBJECTID,FID_WDPA_MPA,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,...,ISO3,marine_area_ha,total_area_ha,Percent_marine_area,MPA,FID_Caribbean_detailed_subregions_mol,ORIG_FID,Area,Subregion,SHAPE
0,1,1,555637442.0,555637442,1,Sombrero Island Nature Reserve Marine Park,Sombrero Island Nature Reserve Marine Park,"Ramsar Site, Wetland of International Importance","Ramsar Site, Wetland of International Importance",International,...,AIA,994.880711,1055.585834,94.249153,yes,8,2,9.948807,Eastern Caribbean,"{""hasZ"": true, ""rings"": [[[-6150002.3972, 2283..."
1,2,2,555705843.0,555705843,1,Little Bay,Little Bay,Marine Park,Marine Park,National,...,AIA,43.801052,70.984313,61.705256,yes,8,2,0.438011,Eastern Caribbean,"{""hasZ"": true, ""rings"": [[[-6123424.997400001,..."
2,3,3,14075.0,14075,1,Shoal Bay and Island Harbour Reefs,Shoal Bay and Island Harbour Reefs,Marine Park,Marine Park,National,...,AIA,1199.883572,1787.212832,67.137139,yes,8,2,11.998836,Eastern Caribbean,"{""hasZ"": true, ""rings"": [[[-6114933.1085, 2244..."
3,4,4,32636.0,32636,1,Prickly Pear Cays and Seal Island Reefs,Prickly Pear Cays and Seal Island Reefs,Marine Park,Marine Park,National,...,AIA,2938.725702,3076.791859,95.512659,yes,8,2,29.387257,Eastern Caribbean,"{""hasZ"": true, ""rings"": [[[-6132163.2213, 2245..."
4,5,5,32638.0,32638,1,Dog Island,Dog Island,Marine Park,Marine Park,National,...,AIA,625.830917,765.341622,81.771447,yes,8,2,6.258309,Eastern Caribbean,"{""hasZ"": true, ""rings"": [[[-6141845.469699999,..."


In [6]:
# Make sure PAs with multiple parcels are only counted once and the areas of the parcels are summed

# group by WDPAID and sum the GIS_AREAs of rows with the same WDPAID (to get a total size of the site)
df_MPA["AREA_SUM"] = df_MPA.groupby("WDPAID")["Area"].transform("sum")
print(df_MPA)

#sort by the Area field, in descending order
df_MPA = df_MPA.sort_values(by = "Area", ascending = False)

#drop rows that have duplicate WDPAIDs, keeping the first duplicate (which is the one with the largest GIS_AREA)
df_MPA_NoDup = df_MPA.drop_duplicates("WDPAID", keep = "first")
print(df_MPA_NoDup)

     OBJECTID  FID_WDPA_MPA       WDPAID   WDPA_PID PA_DEF  \
0           1             1  555637442.0  555637442      1   
1           2             2  555705843.0  555705843      1   
2           3             3      14075.0      14075      1   
3           4             4      32636.0      32636      1   
4           5             5      32638.0      32638      1   
..        ...           ...          ...        ...    ...   
857       858           737     101430.0     101430      1   
858       859           738     342346.0     342346      1   
859       860           739      71021.0      71021      1   
860       861           740      20134.0      20134      1   
861       862           748     167051.0     167051      1   

                                           NAME  \
0    Sombrero Island Nature Reserve Marine Park   
1                                    Little Bay   
2            Shoal Bay and Island Harbour Reefs   
3       Prickly Pear Cays and Seal Island Reefs   


In [9]:
# Filter the results into bins based on MPA area

# Create a list of dictionaries with the results
area_results_dict = [
    {"Bin description": "Total MPA count", "Number of MPAs": df_MPA_NoDup['WDPAID'].nunique()},
    {"Bin description": "Total MPA count with Area < 5 km2", "Number of MPAs": df_MPA_NoDup[df_MPA_NoDup['AREA_SUM'] < 5]['WDPAID'].nunique()},
    {"Bin description": "Total MPA count with Area >= 5 and < 30 km2", "Number of MPAs": df_MPA_NoDup[(df_MPA_NoDup['AREA_SUM'] >= 5) & (df_MPA_NoDup['AREA_SUM'] < 30)]['WDPAID'].nunique()},
    {"Bin description": "Total MPA count with Area >= 30 and < 100 km2", "Number of MPAs": df_MPA_NoDup[(df_MPA_NoDup['AREA_SUM'] >= 30) & (df_MPA_NoDup['AREA_SUM'] < 100)]['WDPAID'].nunique()},
    {"Bin description": "Total MPA count with Area >= 100 km2", "Number of MPAs": df_MPA_NoDup[df_MPA_NoDup['AREA_SUM'] >= 100]['WDPAID'].nunique()}
]

# Create a DataFrame from the list of dictionaries
area_result_df = pd.DataFrame(area_results_dict)

# Print the resulting DataFrame
print(area_result_df)


                                 Bin description  Number of MPAs
0                                Total MPA count             846
1              Total MPA count with Area < 5 km2             313
2    Total MPA count with Area >= 5 and < 30 km2             185
3  Total MPA count with Area >= 30 and < 100 km2             119
4           Total MPA count with Area >= 100 km2             229


In [10]:
# Export results sorted by area to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Adequacy"
table = area_result_df
output_name = "MPA_Carib_int_Adq_4_1_area_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))

In [11]:
# reminder to add a row of total number of WDPAs

# seperate it out by subregion and calculate the percent of MPAs per subregion that are larger tahn 5 km2

# Make pivot table of number of MPAs per subregion
pivot_MPA_subregion = df_MPA_NoDup.pivot_table(
     index = "Subregion",
     values = "WDPAID",
     aggfunc = "count"
).reset_index()
pivot_MPA_subregion.columns = ["Subregion", "Number_MPAs"]
print(pivot_MPA_subregion)

# Make pivot table of number of MPAs greater than 5km2 per subregion

#Filter the data frame
df_MPA_5km = df_MPA_NoDup[df_MPA_NoDup['AREA_SUM'] >5]

#make a pivot table from the filtered df
pivot_MPA_5km = df_MPA_5km.pivot_table(
     index = "Subregion",
     values = "WDPAID",
     aggfunc = "count"
).reset_index()
pivot_MPA_5km.columns = ["Subregion", "Number_MPAs_larger_5km2"]
print(pivot_MPA_5km)

# Merge the pivot tables into a single pivot table. The merge should be based on subregion
size = pd.merge(pivot_MPA_subregion, pivot_MPA_5km, on="Subregion", how="left", validate = "1:1")
print(size)

# Add a row for the total number of unique WDPAIDs with Area > 5
total_MPAs = df_MPA_NoDup['WDPAID'].nunique()
total_MPAs_greater_5_count = df_MPA_NoDup[df_MPA_NoDup['AREA_SUM'] > 5]['WDPAID'].nunique()
total_row = {"Subregion": "Total", "Number_MPAs": total_MPAs, "Number_MPAs_larger_5km2": total_MPAs_greater_5_count}
size = size.append(total_row, ignore_index=True)

#add column for the percent MPAs larger than 5km2 per subregion
size.insert(3, "%MPA_larger_5km2", size["Number_MPAs_larger_5km2"] / size["Number_MPAs"] * 100)
print(size)

                Subregion  Number_MPAs
0                Bahamian           56
1       Eastern Caribbean          115
2                 Florida          144
3        Greater Antilles          202
4                 Guianan           29
5          Gulf of Mexico          108
6      Southern Caribbean           57
7  Southwestern Caribbean           49
8       Western Caribbean           86
                Subregion  Number_MPAs_larger_5km2
0                Bahamian                       37
1       Eastern Caribbean                       48
2                 Florida                       72
3        Greater Antilles                      133
4                 Guianan                       20
5          Gulf of Mexico                       72
6      Southern Caribbean                       40
7  Southwestern Caribbean                       36
8       Western Caribbean                       75
                Subregion  Number_MPAs  Number_MPAs_larger_5km2
0                Bahamian           

C:\Users\bexg\AppData\Local\Temp\ipykernel_8276\2463821511.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  size = size.append(total_row, ignore_index=True)


In [12]:
# Export subregion results excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Adequacy"
table = size
output_name = "MPA_Carib_int_Adq_4_1_subregion_results_v2.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))